In [1]:
import sys
from qwikidata.sparql import return_sparql_query_results
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from SPARQLWrapper import SPARQLWrapper, JSON
import json
from tqdm import tqdm
import re
import requests as r
import pandas as pd
from collections import defaultdict
from libindic import inexactsearch
import pandas as pd
import pickle as pkl
import wptools
sys.path.append('..')
from method2.People_translator import People_translator
translator = People_translator()
from method1.utils import *
from baseline.baseline import baseline_infobox
from method1.get_infobox import method1_infobox

In [5]:
with open('done.pkl' , 'rb') as f:
    done_data = pkl.load(f)

with open('../data-collection/hindi_person_data.jsonl') as f:
    data = f.readlines()
    data = [ json.loads(ent) for ent in data]
wiki_people = []
for entry in data:
    try :
        if entry['en_wikipedia_title'] in done_data: continue
        wiki_people.append([entry['en_wikipedia_title'] , entry['hi_wikipedia_title'] , entry['wd_id']])
    except :
        pass

In [6]:
def get_score(translated_infobox , actual_infobox):
    inst = inexactsearch.InexactSearch()
    markings = defaultdict(list)
    for key in translated_infobox.keys():
        if key in actual_infobox.keys() and key!='image':
            val = inst.compare(translated_infobox[key] , actual_infobox[key])
            if val > 0.70 :
                markings['C'].append(key)
            else : markings['S'].append(key)
    for key in actual_infobox.keys():
        if key not in translated_infobox.keys():
            markings['D'].append(key)
    if len(markings['C']) + len(markings['S']) != 0:
        precision = len(markings['C']) / (len(markings['C']) + len(markings['S']))
    else : precision = 0
    recall = len(markings['C']) / (len(markings['C']) + len(markings['S']) + len(markings['D']))
    print('Precisions :',precision)
    print('Recall :',recall)
    return precision , recall

In [7]:
def parse_infobox(text):
    for key in  ["{{Infobox","{{ज्ञानसन्दूक","{{Geobox","{{ज्ञानसंदूक"]:
        if key in text :
            text =  text.split(key)[1]
            break
    else: 
        return None
    text = text.split("\n")[1:]
    ret = {}
    for line in text:
        if line[:2] == "}}" : break
        if line[0] != '|' : continue
        line = line[1:].split(" = ")
        if len(line) == 1:
            x , y = line[0] , ""
        else:
            x , y = line[0] , line[1]
        ret[x] = y
    else :
        print("Parsing Error")
        assert(False)
    return ret

In [15]:
final_score = []
# 6 for example
for en_name , hi_name , qid in wiki_people:
    try:
        print(en_name , hi_name , qid)
        actual_infobox = parse_infobox(translator.get_page(hi_name , language = 'hi').data['wikitext'])
        if actual_infobox == None: continue
        baseline = baseline_infobox(qid , category='people')
        method2 = translator.get_infobox(en_name , test=True)
        method1 = method1_infobox(qid,'people')
        final_score.append({en_name : [method1 , method2 , baseline , actual_infobox]})
        with open('people_eval.jsonl' , 'a+') as f:
            f.write(json.dumps(final_score[-1] , ensure_ascii = False))
            f.write("\n")
    except Exception as e:
        print(e)
        print(en_name)
print()

Abu Nuwas अबू नुवास Q5670


hi.wikipedia.org (parse) अबू नुवास
hi.wikipedia.org (imageinfo) File:Abu Nuwas.jpg
अबू नुवास (hi) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Abu Nuwas...
  infobox: <dict(12)> name, image, image_size, caption, birth_name...
  pageid: 466584
  parsetree: <str(8902)> <root><template><title>स्रोतहीन</title><p...
  requests: <list(2)> parse, imageinfo
  title: अबू नुवास
  wikibase: Q5670
  wikidata_url: https://www.wikidata.org/wiki/Q5670
  wikitext: <str(6378)> {{स्रोतहीन|date=जून 2015}}{{Infobox writer...
}
en.wikipedia.org (parse) Abu Nuwas
en.wikipedia.org (imageinfo) File:Abu Nuwas.jpg
Abu Nuwas (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Abu Nuwas...
  infobox: <dict(10)> name, image, image_size, caption, birth_name...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Category:A...
  pageid: 654507
  parsetree: <str(36670)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Abu Nuwas
  wikiba

name: अबू नुवास



In [ ]:
with open("final_people.pkl" , "wb") as f:
    pkl.dump(final_score , f)